In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()


#### Load config

In [2]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/human_dura.toml")

config = Config(toml_path=toml_path)
config.BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/human/human_dura/nb"


2023-07-27 15:17:59,208 - hbmep.config - INFO - Verifying configuration ...
2023-07-27 15:17:59,209 - hbmep.config - INFO - Success!


In [3]:
class RectifiedLogistic(Baseline):
    LINK = RECTIFIED_LOGISTIC

    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)
        feature1 = features[1].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]
        n_feature1 = np.unique(feature1).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(3, 10, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(2))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(10))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(2))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(10))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(10))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    with numpyro.plate("n_feature1", n_feature1, dim=-4):
                        """ Priors """
                        a = numpyro.sample(
                            site.a,
                            dist.TruncatedNormal(mu_a, sigma_a, low=0)
                        )
                        b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))

                        L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                        H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))
                        v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                        g_1 = numpyro.sample("g_1", dist.Exponential(1 / 100))
                        g_2 = numpyro.sample("g_2", dist.Exponential(1 / 100))

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature1, feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature1, feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature1, feature0, subject], v[feature1, feature0, subject]) - 1)
                    * jnp.exp(-b[feature1, feature0, subject] * (intensity - a[feature1, feature0, subject])),
                    1 / v[feature1, feature0, subject]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[feature1, feature0, subject] + g_2[feature1, feature0, subject] * (1 / mu)
        )

        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


model = RectifiedLogistic(config=config)

2023-07-27 15:17:59,273 - hbmep.model.baseline - INFO - Initialized model with rectified_logistic link


In [4]:
dir = "/home/vishu/data"

DATASET_REAL = 'real_data/dnc_info_2022-05-26.parquet'
df = pd.read_parquet(os.path.join(dir, DATASET_REAL))

# ind = (df.participant.isin(["scapptio017"]))
# ind &= (df.sc_depth.isin(["epidural", "subdural"]))
# ind &= (df.sc_electrode_configuration.isin(["RC"]))
# ind &= (df.position.isin(["T1_R", "T1_M"]))

ind = (df.participant.isin(["scapptio017"]))
ind &= (df.sc_depth.isin(["epidural"]))
ind &= (df.sc_electrode_configuration.isin(["RC"]))
ind &= (df.position.isin(["T1_R"]))

df = df[ind].copy()
df.reset_index(drop=True, inplace=True)

df[model.intensity] *= 1000

In [5]:
df, encoder_dict = model.load(df=df)

2023-07-27 15:17:59,430 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/human/human_dura/nb
2023-07-27 15:17:59,430 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/human/human_dura/nb
2023-07-27 15:17:59,431 - hbmep.dataset.core - INFO - Processing data ...
2023-07-27 15:17:59,432 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [6]:
model.plot(df=df, encoder_dict=encoder_dict)

2023-07-27 15:17:59,467 - hbmep.dataset.core - INFO - Plotting dataset ...
2023-07-27 15:17:59,738 - hbmep.dataset.core - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/human_dura/nb/dataset.pdf
2023-07-27 15:17:59,738 - hbmep.utils.utils - INFO - func:plot took: 0.27 sec


In [7]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-07-27 15:17:59,861 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-27 15:18:31,514 - hbmep.utils.utils - INFO - func:run_inference took: 31.65 sec


In [8]:
mcmc.print_summary(prob=.95)



                  mean       std    median      2.5%     97.5%     n_eff     r_hat
  H[0,0,0,0]      1.66      1.13      1.53      0.93      2.27    997.96      1.00
  H[0,0,0,1]     13.16      7.75     10.80      3.16     28.75   6310.76      1.00
  H[0,0,0,2]     19.54      7.37     18.15      8.04     34.70   8824.58      1.00
  L[0,0,0,0]      0.49      0.07      0.48      0.36      0.62   8642.79      1.00
  L[0,0,0,1]      0.85      0.53      0.97      0.02      1.64   1525.68      1.00
  L[0,0,0,2]      1.56      0.57      1.57      0.03      2.44   1275.78      1.00
  a[0,0,0,0]      2.59      0.15      2.57      2.36      2.87   5391.03      1.00
  a[0,0,0,1]      1.87      0.90      2.32      0.22      2.95   1259.29      1.00
  a[0,0,0,2]      2.52      0.66      2.67      0.75      3.69    755.69      1.00
  b[0,0,0,0]     12.10      9.29      9.48      0.85     30.92   7744.01      1.00
  b[0,0,0,1]      7.61      6.73      5.62      0.25     20.48   4528.96      1.00
  b

In [9]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-27 15:18:32,334 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-07-27 15:18:35,999 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/human_dura/nb/recruitment_curves.pdf
2023-07-27 15:18:36,000 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 3.67 sec


In [10]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-27 15:18:36,995 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...


2023-07-27 15:18:41,317 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/human_dura/nb/posterior_predictive_check.pdf
2023-07-27 15:18:41,321 - hbmep.utils.utils - INFO - func:render_predictive_check took: 4.33 sec


In [11]:
model.save(mcmc=mcmc)

2023-07-27 15:18:42,271 - hbmep.model.baseline - INFO - Saving inference data ...


2023-07-27 15:18:43,901 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/human_dura/nb/mcmc.nc
2023-07-27 15:18:43,901 - hbmep.model.baseline - INFO - Rendering convergence diagnostics ...
2023-07-27 15:18:46,507 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/human/human_dura/nb/diagnostics.csv
2023-07-27 15:18:46,507 - hbmep.model.baseline - INFO - Evaluating model ...
2023-07-27 15:18:46,760 - hbmep.model.baseline - INFO - ELPD LOO (Log): -173.26
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(
2023-07-27 15:18:46,772 - hbmep.model.baseline - INFO - ELPD WAIC (Log): -172.97
2023-07-27 15:18:46,773 - hbmep.utils.utils - INFO - func:save took: 4.50 sec


In [12]:
a = posterior_samples["a"]

In [13]:
a.shape

(24000, 1, 1, 1, 3)